In [266]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
#tf.config.optimizer.set_jit(False)
from tensorflow import keras
from keras.models import load_model
import random
from sklearn.metrics import mean_absolute_error
#import matplotlib
import os
import pickle
from itertools import zip_longest

from keras.constraints import Constraint
from keras.layers import Layer
from keras import backend as K

import jax
import jax.nn as nn
from jax.random import normal
from jax.experimental import sparse
#from sparse import bcoo_concatenate
from jax.tree_util import tree_map
import jaxlib
import jax.numpy as jnp
import haiku as hk
import optax
from jax import jit
from functools import partial
import functools
from sklearn.decomposition import TruncatedSVD

from scipy.sparse import csr_matrix
from optax import GradientTransformation

from utils import constrained_gradients, StateProbBound, StateProbFolded, Between, between, get_seq_id, get_layer_index
from training import model_training, shuffle_weights, fit_model_grid_jax
from dataloading import load_model_data_jax, resample_training_data_jax
from model_creation import create_model_fn, create_model_jax

In [267]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Define arguments

In [282]:
data_train = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_train.txt'
data_valid = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_valid.txt' 
data_obs = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_all.txt' 
o = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p/ '
e = 100 
l1_regularization_factor = 0 
l2_regularization_factor = 0 
p = 1000 
num_samples= 128,256,512,1024 
learning_rate= 0.001 
num_resamplings= 10 
num_models= 1 
random_seed= 1


In [283]:
args = {
    "data_train": "'/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_train.txt'",
    "data_valid": "/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_valid.txt",
    "data_obs": "/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_all.txt",
    "output_directory": "/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p/",
    "number_additive_traits": 1,
    "l1_regularization_factor": "0.0001,0.001,0.01,0.1",
    "l2_regularization_factor": "0.0001,0.001,0.01,0.1",
    "num_epochs_grid": 10,
    "num_epochs": 10,
    "num_samples": "128,256,512,1024",
    "learning_rate": "0.0001,0.001,0.01,0.1",
    "num_resamplings": 10,
    "early_stopping": False,
    "num_models": 2,
    "random_seed": 1
}

data_train_file = args["data_train"]
data_valid_file = args["data_valid"]
data_obs_file = args["data_obs"]
output_directory = args["output_directory"]
number_additive_traits = args["number_additive_traits"]
num_epochs_grid = args["num_epochs_grid"]
num_epochs = args["num_epochs"]
num_resamplings = args["num_resamplings"]
early_stopping = args["early_stopping"]
num_models = args["num_models"]
random_seed = args["random_seed"]

#Grid search arguments
l1 = [float(i) for i in args["l1_regularization_factor"].split(",")]
l2 = [float(i) for i in args["l2_regularization_factor"].split(",")]
batch_size = [int(i) for i in args["num_samples"].split(",")]
learn_rate = [float(i) for i in args["learning_rate"].split(",")]

In [284]:
output_directory

'/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p/'

# Load model data jax

In [285]:
data_train_file = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_train.txt'

data_valid_file = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_valid.txt' 

data_obs_file = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_all.txt'

In [286]:
#create the rng
random_seed = 42
rng = jax.random.PRNGKey(random_seed)
#Load model data
model_data_jax = load_model_data_jax({
    "train": data_train_file,
    "valid": data_valid_file,
    "obs": data_obs_file
    })

In [287]:
from utils import constrained_gradients, StateProbBound, StateProbFolded, Between, between, get_seq_id, get_layer_index
from training import model_training, shuffle_weights, fit_model_grid_jax
from dataloading import load_model_data_jax, resample_training_data_jax

In [288]:
jax.random.normal(rng, shape=(1,))

DeviceArray([-0.18471177], dtype=float32)

# Resample training data jax

In [289]:
model_data_jax["train"]

{'select': DeviceArray([[0., 1.],
              [0., 1.],
              [1., 0.],
              ...,
              [1., 0.],
              [1., 0.],
              [0., 1.]], dtype=float32),
 'fold': DeviceArray([[1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              ...,
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'bind': DeviceArray([[1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              ...,
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'target': DeviceArray([-0.33598602, -0.8518485 , -1.0957463 , ..., -0.26487023,
              -1.166961  , -0.66433465], dtype=float32),
 'target_sd': DeviceArray([0.11158013, 0.172

In [290]:
#Resample training data
num_resamplings = 1

#create the rng
random_seed = 42
rng = jax.random.PRNGKey(random_seed)

if num_resamplings!=0:
    model_data_jax["train"] = resample_training_data_jax(
        tensor_dict = model_data_jax["train"],
        n_resamplings = num_resamplings,
        rng = rng)

In [291]:
model_data_jax["train"]

{'select': DeviceArray([[0., 1.],
              [0., 1.],
              [1., 0.],
              ...,
              [1., 0.],
              [1., 0.],
              [0., 1.]], dtype=float32),
 'fold': DeviceArray([[1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              ...,
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'bind': DeviceArray([[1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              ...,
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'target': DeviceArray([-0.36398202, -0.92078733, -1.0686252 , ..., -0.63711715,
              -0.7406105 , -0.8554888 ], dtype=float32),
 'target_sd': DeviceArray([0.11158013, 0.172

# Create model Jax

In [292]:
from training import fit_model_grid_jax

In [293]:
num_epochs_grid=50

In [294]:
#Fit model
if len(l1) == 1 and len(l2) == 1 and len(batch_size) == 1 and len(learn_rate) == 1:
    best_params = {
        "num_samples": batch_size[0],
        "learning_rate": learn_rate[0],
        "l1_regularization_factor": l1[0],
        "l2_regularization_factor": l2[0],
        "number_additive_traits": 1
    }
else:
    parameter_grid = [{
        "num_samples": i,
        "learning_rate": j,
        "l1_regularization_factor": k,
        "l2_regularization_factor": l,
        "number_additive_traits": 1
    } for i in batch_size for j in learn_rate for k in l1 for l in l2]

    rng = jax.random.PRNGKey(random_seed)
    rngs = jax.random.split(rng, len(parameter_grid[:3]))
    #print(len(parameter_grid))
    grid_results = [fit_model_grid_jax(params, model_data_jax, num_epochs_grid, rng_key) for params, rng_key in zip(parameter_grid, rngs)]
    print(grid_results)
    #best_params_1 = parameter_grid[[i for i in range(len(grid_results)) if grid_results[i]==min(grid_results)][0]]

    best_params = parameter_grid[np.argmin(grid_results)]

    print("Best: %f using %s" % (min(grid_results), best_params))

num_samples = best_params['num_samples']
learning_rate = best_params['learning_rate']
l1_regularization_factor = best_params['l1_regularization_factor']
l2_regularization_factor = best_params['l2_regularization_factor']


Grid search using {'num_samples': 128, 'learning_rate': 0.0001, 'l1_regularization_factor': 0.0001, 'l2_regularization_factor': 0.0001, 'number_additive_traits': 1}


/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/haiku/_src/data_structures.py:143: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)


epoch done with 0.7139725089073181
epoch done with 0.6315058469772339
epoch done with 0.5704405903816223
epoch done with 0.5250439047813416
epoch done with 0.4910525977611542
epoch done with 0.464991956949234
epoch done with 0.4450560510158539
epoch done with 0.4297783672809601
epoch done with 0.4181148111820221
epoch done with 0.4095253050327301
epoch done with 0.40377748012542725
epoch done with 0.4005716145038605
epoch done with 0.39910459518432617
epoch done with 0.3985155522823334
epoch done with 0.3983135223388672
epoch done with 0.39824819564819336
epoch done with 0.39822229743003845
epoch done with 0.3982141315937042
epoch done with 0.3982124626636505
epoch done with 0.3982105851173401
epoch done with 0.39820927381515503
epoch done with 0.39820823073387146
epoch done with 0.39820680022239685
epoch done with 0.3982059061527252
epoch done with 0.3982042968273163
epoch done with 0.3982035219669342
epoch done with 0.3982018232345581
epoch done with 0.39820075035095215
epoch done wi

KeyboardInterrupt: 

# Fitting final model and testing

In [27]:
#Output model directory
model_directory = os.path.join(output_directory, "whole_model")
#Create output model directory
try:
  os.mkdir(model_directory)
except FileExistsError:
  print("Warning: Output model directory already exists.")

#Output plot directory
plot_directory = os.path.join(output_directory, "plots")
#Create output plot directory
try:
  os.mkdir(plot_directory)
except FileExistsError:
  print("Warning: Output plot directory already exists.")

#Weights directory
weights_directory = os.path.join(output_directory, "weights")
#Create output plot directory
try:
  os.mkdir(weights_directory)
except FileExistsError:
  print("Warning: Output weights directory already exists.")

In [28]:
plot_directory

'/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p/plots'

In [295]:
weights

{'folding_additive_trait': {'w': DeviceArray([[ 0.00970072],
               [ 0.03558933],
               [-0.01015369],
               ...,
               [ 0.04795045],
               [ 0.04954156],
               [ 0.00621062]], dtype=float32),
  'b': DeviceArray([0.], dtype=float32)},
 'folding_additive': {'w': DeviceArray([[-0.2163208]], dtype=float32),
  'b': DeviceArray([0.], dtype=float32)},
 'binding_additive_trait': {'w': DeviceArray([[-3.56119126e-02],
               [-4.08954434e-02],
               [-2.45390031e-02],
               [ 1.59511752e-02],
               [-9.11613181e-02],
               [ 2.89083291e-02],
               [ 2.30520200e-02],
               [-5.46606481e-02],
               [ 3.68875712e-02],
               [-1.00200484e-02],
               [ 1.07338494e-02],
               [-9.18545108e-03],
               [ 7.19583705e-02],
               [-1.07866824e-01],
               [-2.26304512e-02],
               [-3.24731059e-02],
               [ 6.070

In [278]:
num_samples = best_params['num_samples']
learning_rate = best_params['learning_rate']
l1_regularization_factor = best_params['l1_regularization_factor']
l2_regularization_factor = best_params['l2_regularization_factor']
number_additive_traits = best_params['number_additive_traits']

#######################################################################
## BUILD FINAL NEURAL NETWORK ##
#######################################################################

#create the rng
random_seed = 42
rng = jax.random.PRNGKey(random_seed)
rngs = jax.random.split(rng, num=num_models)

model, optimizer = create_model_jax(
    rng=rng,
    learn_rate=learning_rate,
    l1=l1_regularization_factor,
    l2=l2_regularization_factor,
    input_dim_select=model_data_jax['train']['select'].shape[1],
    input_dim_folding=model_data_jax['train']['fold'].shape[1],
    input_dim_binding=model_data_jax['train']['bind'].shape[1],
    number_additive_traits=number_additive_traits
)

weights = model.init(rng, model_data_jax['train']['select'], model_data_jax['train']['fold'], model_data_jax['train']['bind'])
opt_state = optimizer.init(weights)

for model_count in range(num_models):
    print(opt_state)
    #Shuffle model weights
    shuffled_weights = shuffle_weights(rngs[model_count], weights)
    #print(shuffled_weights)
    #Fit the model on best params
    history, model, trained_weights = model_training(model, optimizer, shuffled_weights, opt_state, best_params, model_data_jax, num_epochs_grid, rng)
        
    # Model predictions on observed variants
    model_predictions, folding_additive_trait_layer, binding_additive_trait_layer = model.apply(trained_weights, 
                                    model_data_jax['obs']['select'],
                                    model_data_jax['obs']['fold'],
                                    model_data_jax['obs']['bind']
                                   )
    #save model
    #model.save(os.path.join(model_directory, 'my_model_'+str(model_count)))
    with open(os.path.join(weights_directory, f'weights_{model_count}.pickle'), 'wb') as handle:
        pickle.dump(shuffled_weights, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    #load model
    #model = load_model(os.path.join(model_directory, 'my_model_'+str(model_count)))
    with open(os.path.join(weights_directory, f'weights_{model_count}.pickle'), 'rb') as handle:
        shuffled_weights_reloaded = pickle.load(handle)
        
    #print(history)
    #Plot model performance per epoch
    my_figure = plt.figure(figsize = (8,8))
    plt.plot(np.log(history))
    plt.xlabel('Number of epochs')
    plt.ylabel('Mean Absolute Error (MAE) on testing data')
    plt.show()
    my_figure.savefig(os.path.join(plot_directory, "model_performance_perepoch_"+str(model_count)+".pdf"), bbox_inches='tight')
    

/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/haiku/_src/data_structures.py:143: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)


((ScaleByAdamState(count=DeviceArray(0, dtype=int32), mu={'binding_additive': {'b': DeviceArray([0.], dtype=float32), 'w': DeviceArray([[0.]], dtype=float32)}, 'binding_additive_trait': {'b': DeviceArray([0.], dtype=float32), 'w': DeviceArray([[0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
             [0.],
         

epoch done with 0.6511003971099854
epoch done with 0.5796124935150146
epoch done with 0.5234162211418152
epoch done with 0.4828964173793793
epoch done with 0.45492568612098694


KeyboardInterrupt: 

# Finish the plots below and then move everything to .py folder

In [31]:
# Model predictions on observed variants
model_outputs = model.apply(trained_weights, 
                            model_data_jax['obs']['select'],
                            model_data_jax['obs']['fold'],
                            model_data_jax['obs']['bind']
                           )
print(model_outputs)
prediction, folding_additive_trait_layer_outputs, binding_additive_trait_layer_outputs = model_outputs

folding_additive_trait_df = pd.DataFrame(folding_additive_trait_layer_outputs)
binding_additive_trait_df = pd.DataFrame(binding_additive_trait_layer_outputs)

#Results data frame
dataframe_to_export = pd.DataFrame({
    "seq" : np.array(model_data['obs']['sequence']).flatten(),
    "observed_fitness" : np.array(model_data['obs']['target']).flatten(),
    "predicted_fitness" : model_predictions.flatten(),
    "additive_trait_folding" : folding_additive_trait_df[0],
    "additive_trait_binding" : binding_additive_trait_df[0],
    "training_set" : np.array(model_data['obs']['training_set']).flatten()
})
#Save as csv file
dataframe_to_export.to_csv(os.path.join(output_directory, "predicted_fitness_"+str(model_count)+".txt"),
                           sep = "\t",
                           index = False)


# Save model weights
dataframe_to_export_folding = pd.DataFrame({
    "id" : model_data_jax['obs']['fold_colnames'],
    "folding_coefficient" : [trained_weights["folding_additive"]["w"] for _ in range(len(model_data_jax['obs']['fold_colnames']))]})
dataframe_to_export_binding = pd.DataFrame({
    "id" : model_data_jax['obs']['bind_colnames'],
    "binding_coefficient" : [trained_weights["binding_additive"]["w"] for _ in range(len(model_data_jax['obs']['bind_colnames']))]})

# Save dataframes as csv files 
#Merge
dataframe_to_export = dataframe_to_export_folding.merge(dataframe_to_export_binding, left_on='id', right_on='id', how='outer')
#Save as csv file
dataframe_to_export.to_csv(os.path.join(output_directory, "model_weights_"+str(model_count)+".txt"),
                           sep = "\t",
                           index = False)

# Save remaining model parameters (linear layers)
with open(os.path.join(output_directory, "model_parameters_"+str(model_count)+".txt"), 'w') as f:
    for module_name, module_params in trained_weights.items():
        if module_name in ["folding_additive", "binding_additive"]:
            f.write(module_name.replace("additive", "linear")+"_kernel\n")
            f.write(str(float(module_params["w"]))+"\n")
            f.write(module_name.replace("additive", "linear")+"_bias\n")
            f.write(str(float(module_params["b"]))+"\n")

(DeviceArray([[-0.41215038],
             [-0.4163204 ],
             [-0.9978974 ],
             ...,
             [-0.93618464],
             [-1.0150554 ],
             [-0.41401997]], dtype=float32), DeviceArray([[0.8965007],
             [0.6320977],
             [1.177296 ],
             ...,
             [0.7517024],
             [1.3133274],
             [1.1355388]], dtype=float32), DeviceArray([[-0.759555  ],
             [-0.8795761 ],
             [-0.6599376 ],
             ...,
             [-0.7802873 ],
             [-0.72222495],
             [-1.0715086 ]], dtype=float32))


/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/haiku/_src/data_structures.py:143: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)


NameError: name 'model_data' is not defined

In [296]:
weights

{'folding_additive_trait': {'w': DeviceArray([[ 0.00970072],
               [ 0.03558933],
               [-0.01015369],
               ...,
               [ 0.04795045],
               [ 0.04954156],
               [ 0.00621062]], dtype=float32),
  'b': DeviceArray([0.], dtype=float32)},
 'folding_additive': {'w': DeviceArray([[-0.2163208]], dtype=float32),
  'b': DeviceArray([0.], dtype=float32)},
 'binding_additive_trait': {'w': DeviceArray([[-3.56119126e-02],
               [-4.08954434e-02],
               [-2.45390031e-02],
               [ 1.59511752e-02],
               [-9.11613181e-02],
               [ 2.89083291e-02],
               [ 2.30520200e-02],
               [-5.46606481e-02],
               [ 3.68875712e-02],
               [-1.00200484e-02],
               [ 1.07338494e-02],
               [-9.18545108e-03],
               [ 7.19583705e-02],
               [-1.07866824e-01],
               [-2.26304512e-02],
               [-3.24731059e-02],
               [ 6.070

In [ ]:
shuffled_weights

In [303]:
output_directory = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample50p/'

In [308]:
# Load the CSV file with the saved weights (folding and binding coefficients)
def weights_loading():
    
    weights_filepath = os.path.join(output_directory, "model_weights_"+str(model_count)+".txt")
    weights_dataframe = pd.read_csv(weights_filepath, sep="\t")

    #weights_dataframe = weights_dataframe.dropna(subset=['binding_coefficient'])
    

    # Extract the folding and binding coefficients
    folding_coefficients = weights_dataframe['folding_coefficient'].to_numpy()
    binding_coefficients = weights_dataframe['binding_coefficient'].to_numpy()

    # Load the remaining model parameters (linear layers) from the separate text file
    parameters_filepath = os.path.join(output_directory, "model_parameters_"+str(model_count)+".txt")

    loaded_linear_params = {}
    with open(parameters_filepath, 'r') as f:
        while True:
            module_name = f.readline().strip()
            if not module_name:
                break
            w = float(f.readline().strip())
            f.readline()  # Skip the next line containing the bias name
            b = float(f.readline().strip())
            loaded_linear_params[module_name] = {'w': jnp.array([[w]]), 'b': jnp.array([b])}

    # Create a dictionary with the loaded weights
    loaded_weights = {
        'folding_additive_trait': {
            'w': folding_coefficients.reshape(-1, 1)
        },
        'folding_additive': loaded_linear_params['folding_linear_kernel'],
        'binding_additive_trait': {
            'w': binding_coefficients.reshape(-1, 1)
        },
        'binding_additive': loaded_linear_params['binding_linear_kernel']
    }
    return loaded_weights

# Update the JAX/Haiku model's weights with the loaded weights
#model_params = model.init(jax.random.PRNGKey(0), inputs_select, inputs_folding, inputs_binding)
#model_params = jax.tree_multimap(lambda current, loaded: loaded, model_params, loaded_weights)

In [309]:
loaded_weights

{'folding_additive_trait': {'w': array([[-1.59251881e+00],
         [ 5.39500117e-01],
         [ 1.66388974e-01],
         [ 3.39995474e-01],
         [ 8.52068961e-02],
         [ 3.83542515e-02],
         [ 1.00670859e-01],
         [-1.33205160e-01],
         [ 4.47000228e-02],
         [ 6.44213140e-01],
         [ 4.36202615e-01],
         [ 2.24542260e-01],
         [ 2.61216220e-02],
         [ 3.50446969e-01],
         [ 9.26852345e-01],
         [ 1.24879137e-01],
         [ 6.55783594e-01],
         [-5.78273654e-01],
         [-5.85879162e-02],
         [-4.57141578e-01],
         [ 9.77984786e-01],
         [ 3.31983805e-01],
         [ 1.94089264e-01],
         [ 1.01079738e+00],
         [-2.46983796e-01],
         [ 4.66452181e-01],
         [ 3.04192901e+00],
         [ 3.35610175e+00],
         [ 8.66879344e-01],
         [ 3.36615920e+00],
         [ 1.21622741e+00],
         [ 2.45653892e+00],
         [ 2.95218039e+00],
         [ 3.52295113e+00],
         [ 4.2984

In [301]:
weights

{'folding_additive_trait': {'w': DeviceArray([[ 0.00970072],
               [ 0.03558933],
               [-0.01015369],
               ...,
               [ 0.04795045],
               [ 0.04954156],
               [ 0.00621062]], dtype=float32),
  'b': DeviceArray([0.], dtype=float32)},
 'folding_additive': {'w': DeviceArray([[-0.2163208]], dtype=float32),
  'b': DeviceArray([0.], dtype=float32)},
 'binding_additive_trait': {'w': DeviceArray([[-3.56119126e-02],
               [-4.08954434e-02],
               [-2.45390031e-02],
               [ 1.59511752e-02],
               [-9.11613181e-02],
               [ 2.89083291e-02],
               [ 2.30520200e-02],
               [-5.46606481e-02],
               [ 3.68875712e-02],
               [-1.00200484e-02],
               [ 1.07338494e-02],
               [-9.18545108e-03],
               [ 7.19583705e-02],
               [-1.07866824e-01],
               [-2.26304512e-02],
               [-3.24731059e-02],
               [ 6.070